# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [125]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

In [126]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [127]:
data_0.describe()

,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.72,128.56
max,2.36,1.34,16.00,185.36


In [128]:
data_1.describe()

,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,1.14,-4.80,2.49,68.83
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95


In [129]:
data_2.describe()

,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03


In [130]:
data_0.info()
data_1.info()
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

In [131]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04


In [132]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77


In [133]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60


Удалим колонку 'id' из датасетов, т.к. она не нужна при обучении моделей.

In [134]:
data_0 = data_0.drop('id', axis=1)
data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)

## Обучение и проверка модели

Напишем функцию для обучения модели по датасету.

In [135]:
def predict_data (data):
    # Разделим данные на выборки
    df_train, df_valid = train_test_split(data, test_size=0.25, random_state=12345)
    
    # Сохраним признаки в отдельных переменных
    features_train = df_train.drop('product', axis=1)
    target_train = df_train['product']

    features_valid = df_valid.drop('product', axis=1)
    target_valid = df_valid['product']
    
    # Приведем численные признаки к одному масштабу
    numeric = ['f0','f1','f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    # Обучим модель линейной регрессии 
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = pd.Series(model.predict(features_valid))
    
    # упорядочим индексы у target_valid
    target_valid.index = predicted_valid.index
    
    return target_valid, predicted_valid    

Получим предсказания модели

In [136]:
target_0, predicted_0 =  predict_data (data_0)
target_1, predicted_1 =  predict_data (data_1)
target_2, predicted_2 =  predict_data (data_2)

Рассчитаем Cредний запас предсказанного сырья и RMSE по регионам

In [137]:
print("Регион 0")
print("RMSE =", mean_squared_error(target_0, predicted_0) ** 0.5)
print("Cредний запас предсказанного сырья =", predicted_0.mean())

Регион 0
RMSE = 37.5794217150813
Cредний запас предсказанного сырья = 92.59256778438038


In [138]:
print("Регион 1")
print("RMSE =", mean_squared_error(target_1, predicted_1) ** 0.5)
print("Cредний запас предсказанного сырья =", predicted_1.mean())

Регион 1
RMSE = 0.8930992867756158
Cредний запас предсказанного сырья = 68.728546895446


In [139]:
print("Регион 2")
print("RMSE =", mean_squared_error(target_2, predicted_2) ** 0.5)
print("Cредний запас предсказанного сырья =", predicted_2.mean())

Регион 2
RMSE = 40.02970873393434
Cредний запас предсказанного сырья = 94.96504596800489


Регион 1 имеет значительно меньший RMSE, чем остальные. Но и средние запасы сырья у него тоже меньше.

## Подготовка к расчёту прибыли

Заведем константы согласно условий задачи.

In [140]:
PROFIT_PER_PROD = 450000
N_TOTAL = 500
N_BEST = 200
BUDGET = 10000000000

Рассчитаем минимальное среднее количество продукта в месторождениях региона, достаточное для разработки

In [141]:
min_prod = BUDGET/N_BEST/PROFIT_PER_PROD
print("минимальное среднее количество продукта в месторождениях региона, достаточное для разработки =",min_prod)

минимальное среднее количество продукта в месторождениях региона, достаточное для разработки = 111.11111111111111


Среднее значение, достаточное для разработки выше средних по регионам. Посмотрим как поменяется ситуация, когда будем выбирать 200 лучших месторождений по региону

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [142]:
def profit_data (target, predict):
    predict = predict.sort_values(ascending = False)
    profit = target[predict.head(N_BEST).index].sum() * PROFIT_PER_PROD - BUDGET
    return profit

## Расчёт прибыли и рисков 

Создадим функцию profit_calc которая будет считать и выводить на экран:

- Среднюю прибыль
- Границы 95%-й доверительного интервала
- Риск получения убытка

In [143]:
def profit_calc (target, predict):
    state = np.random.RandomState(12345)

    values = []
    for i in range(1000):
        target_subsample = target.sample(n=N_TOTAL, replace=True, random_state=state)
        probs_subsample = predict[target_subsample.index]
        values.append(profit_data(target_subsample,probs_subsample))


    values = pd.Series(values)
    lower =  values.quantile(0.025)
    upper = values.quantile(0.975)

    mean = values.mean()
    print("Средняя прибыль:", '{:,.0f}'.format(mean))
    print("95%-й доверительный интервал", '{:,.0f} - {:,.0f}'.format(lower, upper))
    print("Риск получения убытка", values[values < 0].count()/values.count())
    return

### Рассчитаем показатели для каждого региона.

#### Показатели по Региону_0

In [144]:
profit_calc (target_0, predicted_0)

Средняя прибыль: 600,735,244
95%-й доверительный интервал 12,948,331 - 1,231,163,606
Риск получения убытка 0.02


#### Показатели по Региону_1

In [145]:
profit_calc (target_1, predicted_1)

Средняя прибыль: 665,241,058
95%-й доверительный интервал 157,988,481 - 1,197,641,587
Риск получения убытка 0.003


#### Показатели по Региону_2

In [146]:
profit_calc (target_2, predicted_2)

Средняя прибыль: 615,559,723
95%-й доверительный интервал -12,218,495 - 1,230,644,474
Риск получения убытка 0.03


Добавим таблицу по регионам.

In [147]:
data_res = pd.DataFrame(
    {
        'region': [
            0,
            1,
            2,
        ],
        'avg_profit': [
            600735244.,
            665241058.,
            615559723.,
        ],
        'lower_95%': [
            12948331.,
            157988481.,
            -12218495.,
        ],
        'upper_95%': [
            1231163606.,
            1197641587.,
            1230644474.,
        ],
        'risk_of_loss': [
            '0.02',
            '0.003',
            '0.03',
        ],
    }
)

pd.options.display.float_format = '{:,.2f}'.format

data_res

,region,avg_profit,lower_95%,upper_95%,risk_of_loss
0,0,"600,735,244.00","12,948,331.00","1,231,163,606.00",0.02
1,1,"665,241,058.00","157,988,481.00","1,197,641,587.00",0.003
2,2,"615,559,723.00","-12,218,495.00","1,230,644,474.00",0.03


## Общий вывод

Для разработки предлагаю Регион_1 по следующим причинам:
1. Наибольшая средняя прибыль = 665 млн.руб.
2. Наибольшая нижняя граница 95%-го доверительного интервала = 158 млн.руб.
3. Наименьший риск получения убытка = 0.3%